# Precomputed

This notebook contains functions for a precomputed bounding surface model. The functions are developed and maintained by Justin Bonus (University of Washington).

Use ``%run YOURPATH/Precomputed.ipynb`` at the start of your notebook to import these functions.

In [1]:
def kappaLayer(res_x, res_y, minKappa, maxKappa, Stress0):
    #===============================================================================================
    # Kappa Matrix layer creator for Borjas & Amies 1994 bounding surface constitutive model
    #===============================================================================================
    # Author: Justin K. Bonus, University of Washington
    # Created: July 21, 2019
    #
    # This function creates a complete matrix of all possible kappa values relative to 
    # the last point of unloading.
    # In construction it is agnostic to the active stress point angular position and bounding 
    # surface but in use both must be known to properly access the matrix.
    #
    # One layer of the kappa matrix is made with this function, meaning it only defines
    # kappa contours for a single unloading point.
    #
    # NOTE: This method is highly dependent on input resolution. High resolution will create a
    # profile that is extremely close to the true function, but at low resolutions it will appear
    # to be fairly discontinuous around the centerline and circles will be rough.
    #
    # Workflow: 
    # 0.) Consider centerline symetry (optional, by default it is not used)
    # 1.) Beginning in cartesian coordinates, create zeros matrix of the desired size
    # 2.) Create a ring to represent the bounds of the deviatoric space, set temporary -1. value
    # 3.) Fully evaluate the kappa values in each cell of the centerline, utilizing 1D equation
    # 4.) For each point, find another point opposite the kappa singularity that are equal/near-equal
    # 5.) Define circle contour radii and midpoints from the matched points
    # 6.) Create the kappa value contours with the ring function
    # 7.) Insert new contours onto main matrix, careful to not overwrite/underwrite (thresholds)
    # 8.) Convert to polar coordinates for compactness
    #===============================================================================================
    #import numpy as np
    from skimage.draw import circle
    from skimage.draw import circle_perimeter
    
    radius = int(np.floor(res_x/2))
    cen_x = int(np.floor(res_x/2))
    cen_y = int(np.floor(res_y/2))
    
    def ring(res_x, res_y, midpoint, radius, kappaSouth, kappaNorth):
        # Use Scikit to create efficient circle coordinates
        from skimage.draw import circle_perimeter
        cen_x = int(np.floor(res_x/2))
        cen_y = int(np.floor(res_y/2))
        rr, cc = circle_perimeter(midpoint, cen_x, radius) # Creates coordinates
        donut = np.zeros((res_y, res_x)) # Create empty matrix
        north = np.array([[-1],[0]]) # North unit vector
        for yy, xx in zip(rr,cc):
            yy = yy - midpoint
            xx = xx - cen_x
            index = np.array([[yy],[xx]])
            angle = np.abs(angleIndex(north, index)) # (0 to pi)
            # Weight kappa value at coordinate by angle to south
            donut[rr, cc] = kappaSouth - (angle/np.pi)*(kappaSouth - kappaNorth)
        return donut
        
    cart = np.zeros((res_y,res_x)) # Holds values as they are sequentially added in each \kappa contour
    over = np.zeros((res_y,res_x)) # Holds number of times a cart cell has been overwritten, used for averaging
    
    # Define area within bounding surface with -1 values, outside with 0 values
    rr, cc = circle(cen_y, cen_x, radius)
    cart[rr,cc] = -1
    
    # Define the bounding surface perimeter with -2 values
    rr, cc = circle_perimeter(cen_y,cen_x, radius)
    cart[rr,cc] = -2
    
    # Apply maxKappa value to unloading point, minKappa value to extents of centerline 
    cart[Stress0[0],Stress0[1]] = maxKappa
    cart[0, cen_x], cart[res_y-1, cen_x] = minKappa, minKappa

    # Solve kappa for each cell on the vertical centerline using 1D equation
    # \kappa = (|\sigma'_hat|-|\sigma'|)/(|\sigma'-\sigma'_0|)
    for j in range(0, res_y):
        if j < cen_y and j > Stress0[0]:
            cart[j,cen_x] = (cen_y + np.abs(cen_y-j))/np.abs((cen_y-j)-(cen_y-Stress0[0]))
        else:
            cart[j,cen_x] = (cen_y - np.abs(cen_y-j))/np.abs((cen_y-j)-(cen_y-Stress0[0]))
    
    #Evaluate points on center-line, below Stress0 y-center
    for ja in range(Stress0[0]+1,res_y):
        hold = [] 

        # Evaluate points on center-line, above Stress0 y-center
        for jb in range(1,Stress0[0]):
            # Contain kappa differentials in a list, we want the smallest one 
            # This will allow us to define to points of equipotential
            hold.append(np.abs(cart[jb,cen_x] - cart[ja,cen_x]))  
        if len(hold) != 0:
            jb = hold.index(min(hold)) #jb the best matches ja index
            midpoint = int(np.round((ja+jb)/2)) #Finds index of kappa contour midpoint 
            rad = int(np.abs(midpoint-ja)) #Determines radius of contour
            kappaSouth = cart[ja,cen_x]
            kappaNorth = cart[jb,cen_x]
        else:
            # When at bounding surface?
            midpoint = int(np.round((ja)/2))
            rad = int(np.abs(midpoint-ja)) 
            kappaSouth = cart[ja,cen_x]
            kappaNorth = cart[ja,cen_x]
        
        
        
        donut = ring(res_x, res_y, midpoint, rad, kappaSouth, kappaNorth)
        
        #Insert all kappa contours into main array 
        for x in range(0,res_x):
            #Prevent overwriting of vertical kappa_line
            if x == Stress0[1]:
                continue
            for y in range(0,res_y):
                if donut[y,x] == 0:
                    # Skip zero values   
                    continue
                if cart[y,x] == -1:
                    # Cell is 'empty', write \kappa value to it
                    cart[y,x] = donut[y,x]
                elif cart[y,x] > 0:
                    # Weighted overwriting
                    factor = over[y,x] + 2
                    cart[y,x] = (donut[y,x]*1)/factor + (cart[y,x]*(factor-1))/factor
                    over[y,x] = over[y,x]+1
                    
    #============= POST-PROCESS ============= 
    # Reapply bounding surface and \kappa singularity
    #rr, cc = circle_perimeter(cen_y,cen_x, radius)
    cart[rr,cc] = minKappa
    cart[Stress0[0],Stress0[1]] = maxKappa
    # Horizontal linear interpolation for empty cells
    fill = True
    if fill:
        xp = np.asarray(np.nonzero(cart.ravel() > 0))
        fp = cart[np.nonzero(cart > 0)]
        fp = np.asarray(fp).reshape(1,fp.shape[0])
        x  = np.nonzero(cart.ravel() == -1)
        cart[np.nonzero(cart == -1)] = np.interp(x, xp[0,:], fp[0,:])
      
    return cart

In [2]:
def kappaMatrix(res_x, res_y, minKappa, maxKappa):
    #===============================================================================================
    # Kappa Matrix creator for Borjas & Amies 1994 bounding surface constitutive model
    #===============================================================================================
    # Author: Justin K. Bonus, University of Washington
    # Created: July 21, 2019
    #
    # -----INPUT
    # res_x = X indice resolution (odd #)
    # res_y = Y indice resolution (odd #)
    # minKappa = Minimum \kappa value to apply at bounding surface
    # maxKappa = Maximum \kappa value to apply at singularity
    # -----OUTPUT
    # kappa_matrix: Populated 3D matrix for the desired input settings
    #===============================================================================================
    import numpy as np
    from IPython.display import clear_output
    #Center of space
    cen_x = int(np.floor(res_x/2))
    cen_y = int(np.floor(res_y/2))
    
    kappa_matrix = np.empty((res_y, res_x, cen_y+1)) # Initialize first layer dimensions
    
    # Iteratively create and append \kappa layers to form \kappa matrix
    for layer in range(0,int(np.floor(res_y/2))+1):
        # Varies unloading point on north centerline
        Stress0 = np.array([ cen_y - layer, cen_x ]) 
        cart = kappaLayer(res_x, res_y, minKappa, maxKappa, Stress0) # Retrieves associated \kappa layer
        kappa_matrix[:,:,layer] = cart
        clear_output(wait=True)
        print('Layer:', layer, 'of ', int(res_y/2))                
    print('Kappa Matrix Shape:',kappa_matrix.shape)

    return kappa_matrix

In [1]:
#================= PLOTTING KAPPA LAYER - STATIC =================
def kappaLayer_plot():
    #%matplotlib inline
    import numpy as np
    import matplotlib
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D

    #Center of space, requires odd number resolution for a true center
    cen_x = int(np.floor(res_x/2))
    cen_y = int(np.floor(res_y/2))

    #Point of last unloading by index, will need a mapping algorithm to relate to deviatoric stresses
    #Unloading x index should always be centered in space, may require stress rotation in driver 
    #Do not have the y-index exceed cen_y, as it is redundant
    Stress0 = np.array([ cen_y - int(cen_y/2) + 1, cen_x ])
    #Stress0 = np.array([ 0, cen_x ])
    
    cart = kappaLayer(res_x, res_y, minKappa, maxKappa, Stress0)
    #cart = kappaM[:,:,Stress0[0]]
    kappaline = cart[:,cen_x]
    midpoints = -(kappaline*(cen_y - Stress0[0]))/(kappaline + 1) + cen_y
    centerline = np.linspace(0,res_y,res_y)
    
    fig, (ax1, ax2) = plt.subplots(1,2, sharey=True, gridspec_kw={'wspace': 0})

    #Plot \kappa heatmap in deviatoric index view
    extent = [0,res_x,res_y,0]
    ax1.set_xlabel('X-Index',fontsize=14)
    ax1.set_ylabel('Y-Index',fontsize=14)
    ax1.set_ylim([res_y,0])
    kappa_display = ax1.imshow(cart,cmap='inferno',interpolation='nearest', norm=matplotlib.colors.LogNorm(), extent=extent)
    plt.colorbar(kappa_display)

    #Plot centerline \kappa distribution in logspace
    ax2.set_xlabel('Centerline $\kappa$ Value',fontsize=14)
    ax2.set_xscale('log')
    ax2.set_xlim([minKappa,maxKappa])
    ax2.set_ylim([res_y,0])
    ax2.plot(kappaline, centerline, color = 'black', antialiased=True)
    ax2.plot(np.sort(kappaline), np.sort(midpoints)[::-1], color = 'red', antialiased=True)
    # fit subplots and save fig
    fig.tight_layout()
    fig.set_size_inches(w=10,h=5)
    fig_name = 'kappa_layer_visual'
    fig.savefig(fig_name)

    plt.show()

#===================================================================

In [2]:
def kappaLayer_surf():
    #%matplotlib inline
    import matplotlib
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D

    cen_x = int(np.floor(res_x/2))
    cen_y = int(np.floor(res_y/2))

    Stress0 = np.array([ cen_y - int(cen_y/2), cen_x ])
    #Stress0 = np.array([ cen_y, cen_x ])

    cart = kappaM[:,:,Stress0[0]]
    x = range(res_x)
    y = range(res_y)

    for i in range(0,res_x):
        for j in range(0,res_y):
            if cart[j,i] <=0:
                cart[j,i] = minKappa            

    cart_log = np.log(cart)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    X, Y = np.meshgrid(x, y)  # `plot_surface` expects `x` and `y` data to be 2D,
    surf = ax.plot_surface(X, Y, cart_log, cmap='inferno', antialiased=True, rstride=1, cstride=1, alpha=1)
    surf
    fig.colorbar(surf,shrink=0.5,aspect=20)

    #Make the panes transparent
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))

    #Make the grid lines transparent
    ax.set_axis_off()
    ax.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    ax.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    ax.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    fig.set_size_inches(w=10,h=10)
    plt.tight_layout
    plt.show()

In [3]:
#================= PLOTTING HARDENING LAYER - STATIC =================
def hardeningLayer_plot():
    #%matplotlib inline
    import numpy as np
    import matplotlib
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D

    #Center of space, requires odd number resolution for a true center
    cen_x = int(np.floor(res_x/2))
    cen_y = int(np.floor(res_y/2))
    

    #Point of last unloading by index, will need a mapping algorithm to relate to deviatoric stresses
    #Unloading x index should always be centered in space, may require stress rotation in driver 
    #Do not have the y-index exceed cen_y, as it is redundant
    
    #Stress0 = np.array([ cen_y-int(cen_y/2), cen_x ])
    #kappaM = kappaMatrix(res_x, res_y, minKappa, maxKappa)
    #cart = hardeningMatrix(hh,mm,kappaM)
    #kappaM = kappaMatrix(res_x, res_y, minKappa, maxKappa)
    Stress0 = np.array([cen_y-int(cen_y/2)-1,cen_x])
    #Stress0 = np.array([cen_y,cen_x])
    cart = hardeningM[:,:,Stress0[0]]
    
    hardeningline = cart[:,cen_x]
    centerline = np.linspace(0,res_y,res_y)
    fig, (ax1, ax2) = plt.subplots(1,2, sharey=True, gridspec_kw={'wspace': 0})

    #Plot \kappa heatmap in deviatoric index view
    extent = [0,res_x,res_y,0]
    ax1.set_xlabel('X-Index',fontsize=14)
    ax1.set_ylabel('Y-Index',fontsize=14)
    ax1.set_ylim([res_y,0])
    hardening_display = ax1.imshow(cart,cmap='cividis',interpolation='nearest', norm=matplotlib.colors.LogNorm(), extent=extent)
    plt.colorbar(hardening_display)

    #Plot centerline \kappa distribution in logspace
    ax2.set_xlabel('Centerline H\' Value',fontsize=14)
    ax2.set_xscale('log')
    ax2.set_xlim([minKappa,int(np.amax(cart))])
    ax2.set_ylim([res_y,0])
    ax2.plot(hardeningline, centerline, color = 'black', antialiased=True)

    # fit subplots and save fig
    fig.tight_layout()
    fig.set_size_inches(w=10,h=5)
    fig_name = 'hardening_layer_visual'
    fig.savefig(fig_name)

    plt.show()

    #===================================================================

In [4]:
#================= PLOTTING PSI LAYER - STATIC =================
def psiLayer_plot():
    #%matplotlib inline
    import numpy as np
    import matplotlib
    import matplotlib.pyplot as plt

    #Center of space, requires odd number resolution for a true center
    cen_x = int(np.floor(res_x/2))
    cen_y = int(np.floor(res_y/2))

    #Point of last unloading by index, will need a mapping algorithm to relate to deviatoric stresses
    #Unloading x index should always be centered in space, may require stress rotation in driver 
    #Do not have the y-index exceed cen_y, as it is redundant
    Stress0 = np.array([int(cen_y/2),cen_x])
    CurStress = np.array([cen_y,cen_x])
    
    Stress0 = np.array([cen_y, cen_x]) #ERROR: 2G point never reaches bounding surface! cen_y limit
    CurStress = np.array([cen_y,cen_x]) 
    layer = Stress0[0]*(res_y - 1) + CurStress[0] #int(res_y/2) + n*res_x gives 2G max value
    
    subG = 25 # SubGroup : 0 to cen_y : (cen_y+1) total : Moves from center to top
    curS = 25 # CurStress : 0 to res_y-1 : res_y total : Moves from top to bottom
    layer = (subG)*(res_y) + (curS) # Layer to access
    
    cart = psiM[:,:,layer] # Load layer in RAM
    print('layer:',layer+1, 'of', psiM.shape[2])
    psiline = cart[:,cen_x]
    centerline = np.linspace(0,res_y,res_y)
    fig, (ax1, ax2) = plt.subplots(1,2, sharey=True, gridspec_kw={'wspace': 0})

    #Plot \kappa heatmap in deviatoric index view
    extent = [0,res_x,res_y,0]
    ax1.set_xlabel('X-Index',fontsize=14)
    ax1.set_ylabel('Y-Index',fontsize=14)
    ax1.set_ylim([res_y,0])
    # Set values outside bounding surface to 'white' for visual puposes
    cart = np.ma.masked_where(cart < minKappa, cart)
    cmap = plt.cm.viridis
    cmap.set_bad(color='white')
    psi_display = ax1.imshow(cart,cmap=cmap,interpolation='nearest', extent=extent) #norm=matplotlib.colors.LogNorm()
    plt.colorbar(psi_display)

    #Plot centerline \kappa distribution in logspace
    ax2.set_xlabel('Centerline $\psi$ Value',fontsize=14)
    #ax2.set_xscale('log')
    ax2.set_xlim([0, 2*G])
    ax2.set_ylim([res_y,0])
    ax2.plot(psiline, centerline, color = 'black', antialiased=True)

    # fit subplots and save fig
    fig.tight_layout()
    fig.set_size_inches(w=10,h=5)
    fig_name = 'psi_layer_visual'
    fig.savefig(fig_name)

    plt.show()

    #===================================================================

In [5]:
def psiLayer_surf():
    #%matplotlib inline
    import matplotlib
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D

    cen_x = int(np.floor(res_x/2))
    cen_y = int(np.floor(res_y/2))

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x = range(res_x); y = range(res_y)
    X, Y = np.meshgrid(x, y)  # `plot_surface` expects `x` and `y` data to be 2D,

    # Layer selection
    Stress0 = np.array([0,cen_x])
    CurStress = np.array([cen_y,cen_x])
    layer = Stress0[0]*(res_y - 1) + CurStress[0]
    subG = 25 # SubGroup : 0 to cen_y : (cen_y+1) total : Moves from center to top
    curS = 25 # CurStress : 0 to res_y-1 : res_y total : Moves from top to bottom
    layer = (subG)*(res_y) + (curS) # Layer to access
    cart = psiM[:,:,layer]

    # Create surface
    surf = ax.plot_surface(X, Y, cart, cmap='viridis', antialiased=True, rstride=1, cstride=1, alpha=1)
    surf

    #Make the panes transparent
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))

    #Make the grid lines transparent
    ax.set_axis_off()
    ax.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    ax.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    ax.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)

    fig.colorbar(surf,shrink=0.5,aspect=20)
    fig.set_size_inches(w=10,h=10)
    plt.tight_layout
    plt.show()

In [6]:
def psiSearch_plot():
    # Visually demonstrate psiSearch() from \Pi-plane perspective
    # Minor issues in 2nd and 4th quadrant dStrain 
    # (search 1 too far and too few respectively)
    #%matplotlib inline
    from skimage.draw import circle_perimeter

    R = 0.1
    res_x = 101
    res_y = 101
    cen_x = int(res_x/2)
    cen_y = int(res_y/2)

    # Instead of using Stress0, for now lets assign a \kappa
    kappa = 1e-1


    # Don't allow normS(dStrain) > R
    dStrain = np.array([[0.00],[.0004],[.0001]])
    CurStress = np.array([[0.02],[0.0],[0.01]])

    dev_dStrain = dev(dStrain)
    norm_dev_dStrain = normS(dev_dStrain)

    midpoint = int((kappa * R)/(kappa + 1)) # midpoint of contour on centerline
    midpoint = np.abs(midpoint - cen_y) # UL notation
    print('midpoint:',midpoint)

    south = np.array([[-np.sqrt(2/3)],[np.sqrt(1/6)],[np.sqrt(1/6)]])
    east = np.array([[0],[np.sqrt(1/2)],[-np.sqrt(1/2)]])
    angle = angleNormal(south, dev(CurStress)) # How much to rotate dStrain clockwise by, depends on CurStress
    theta = angleNormal(east, dev(CurStress))
    if theta < np.pi/2:
        angle = -angle
    print('Angle:',round(angle*180/np.pi,2))
    if np.isnan(angle):
        angle = 0

    southAngle = angle
    northAngle = 90. - angle

    factor = 1.
    if norm_dev_dStrain > R:
        factor = R / norm_dev_dStrain
    dir_dev_dStrain = factor * dev_dStrain

    southDStrain = quatHydroRotator(dir_dev_dStrain, -southAngle) # Rotated 3x1 dStrain to maintain relative distance to rotCurStress
    northDStrain = quatHydroRotator(dir_dev_dStrain, -northAngle)
    southCurStress = quatHydroRotator(dev(CurStress), -southAngle) # 3x1 CurStress rotated to \Pi-plane centerline
    northCurStress = quatHydroRotator(dev(CurStress), -northAngle)

    indexCurStress = stressToIndex(CurStress, res_x, res_y, R)
    indexSouthCurStress = stressToIndex(southCurStress, res_x, res_y, R) # UL-centered indexed CurStress, on centerline
    indexNorthCurStress = stressToIndex(northCurStress, res_x, res_y, R)

    img = np.zeros((res_y, res_x), dtype=np.uint8)

    xb, yb = circle_perimeter(cen_y,cen_x,cen_y) 

    img[xb, yb] = 12 # Bounding Surface
    cen_x = int(np.floor(res_x/2))
    img[:,cen_x] = 11
    img[cen_y,:] = 11

    rr, cc = psiSearch(dir_dev_dStrain, indexCurStress, res_x, res_y, R)
    img[rr, cc] = 9
    img[rr[0], cc[0]] = 8 # Line Start
    img[rr[len(rr)-1], cc[len(cc)-1]] = 10 # Line End

    rrSouth, ccSouth = psiSearch(southDStrain, indexSouthCurStress, res_x, res_y, R)
    img[rrSouth, ccSouth] = 3 # Search Line
    img[rrSouth[0], ccSouth[0]] = 2 # Line Start
    img[rrSouth[len(rrSouth)-1], ccSouth[len(ccSouth)-1]] = 4 # Line End

    rrNorth, ccNorth = psiSearch(northDStrain, indexNorthCurStress, res_x, res_y, R)
    img[rrNorth, ccNorth] = 6
    img[rrNorth[0], ccNorth[0]] = 5 # Line Start
    img[rrNorth[len(rrNorth)-1], ccNorth[len(ccNorth)-1]] = 7 # Line End

    curRad = int(np.round(np.sqrt((rrSouth[0] - rrNorth[0])**2 + (ccSouth[0] - ccNorth[0])**2))/2)
    xc, yc = circle_perimeter(cen_y, cen_x, curRad)
    xc = xc #+ (indexCurStress[0] - cen_x)
    yc = yc #+ (indexCurStress[1] - cen_y)
    img[yc,xc] = 13

    plt.imshow(img, cmap = 'hot')
    plt.show()

In [7]:
def rotateDemo(Stress0, CurStress):
    #================================================================
    # Justin Bonus, September 30, 2019
    # Create stress state that rotates to N-S for animation purposes
    # (3,1) principal normal stress inputs
    # Output: (X,) time, (3,X) rotStress0, (3,X) rotCurStress 
    #================================================================
    Stress0 = dev(Stress0)
    CurStress = dev(CurStress)
    north = dev(np.array([[np.sqrt(3/2)],[0],[0]])) # Deviatoric north unit-vector
    east = dev(np.array([[0],[np.sqrt(1/2)],[-np.sqrt(1/2)]]))
    
    angleNorth = angleNormal(north, Stress0) # Total north angle
    angleEast = angleNormal(east, Stress0)
    if np.isnan(angleNorth):
        angleNorth = 0
    if np.isnan(angleEast):
        angleEast = 0
    if angleEast < np.pi/2:
        angleNorth = -angleNorth # Rotate correct direction
    northSteps = 12
    northStress0 = Stress0 # Initial
    northCurStress = CurStress
    for i in range(0,northSteps):
        angleStep = angleNorth/12
        Stress0 = quatHydroRotator(Stress0, angleStep)
        CurStress = quatHydroRotator(CurStress, angleStep)
        northStress0 = np.append(northStress0,Stress0, axis=1)
        northCurStress = np.append(northCurStress,CurStress, axis=1)        
    
    # Find contour center
    #i = cen_x
    #last = northStress0.shape[1] - 1
    #indexNorthStress0 = stressToIndex(northStress0[:,last], res_x, res_y, R) # Correct?
    #indexNorthCurStress = stressToIndex(northCurStress[:,last], res_x, res_y, R)
    #k = int(indexNorthStress0[1])
    #j = int(indexNorthCurStress[1])
    #kappa = int(kappaM[j,i,k]) # Correct?
    #midpoint = 
    #midpoint = np.array([[midpoint],[],[]])
    midpoint = northStress0[:,-1]/2 # Assumed value, could be found with kappaMatrix
    adjNorthCurStress = (northCurStress[:,-1] - midpoint) # Corrected for contour center
    adjNorthCurStress = adjNorthCurStress.reshape(3,1)

    angleSouth = angleNormal(-north, adjNorthCurStress)
    angleEast = angleNormal(east, adjNorthCurStress)
    if np.isnan(angleSouth):
        angleSouth = 0
    if np.isnan(angleEast):
        angleEast = 0
    if angleEast > np.pi/2:
        angleSouth = -angleSouth
    southSteps = 20 # Frames for south rotation
    southNorthStress0 = northStress0 # Initialize
    southNorthCurStress = northCurStress # Initialize
    for i in range(0,southSteps):
        angleStep = angleSouth/southSteps # Rotation per frame
        southNorthStress0 = np.append(southNorthStress0, northStress0[:,-1].reshape(3,1), axis=1) # Maintain northStress0
        
        # Shift hydrostatic axis for rotation around contour center
        adjNorthCurStress = quatHydroRotator(adjNorthCurStress, angleStep)
        adjAdjNorthCurStress = np.zeros((3,1))
        adjAdjNorthCurStress[0] = adjNorthCurStress[0] + midpoint[0]
        adjAdjNorthCurStress[1] = adjNorthCurStress[1] + midpoint[1]
        adjAdjNorthCurStress[2] = adjNorthCurStress[2] + midpoint[2]

        southNorthCurStress = np.append(southNorthCurStress, adjAdjNorthCurStress, axis=1) # Rotate northCurStress south
    
    buffer = 6 # Extra frames at end
    for i in range(0,buffer):
        southNorthStress0 = np.append(southNorthStress0, southNorthStress0[:,-1].reshape(3,1), axis=1)
        southNorthCurStress = np.append(southNorthCurStress, southNorthCurStress[:,-1].reshape(3,1), axis =1)
        
    rotStress0 = southNorthStress0
    rotCurStress = southNorthCurStress
    time = np.linspace(0,1,rotCurStress.shape[1]+1) # Time increments
    
    return time, rotStress0, rotCurStress

In [10]:
def angleNormal(u,v):
    # Based on Olivier Verdier's script, May 13 2010
    # Return angle between 3D principal normal stress vectors
    # (0,180) range
    # Input dev versions of vectors if you want angle on \Pi-plane
    # Lacks angle sign, Nan return if zero vector used
    u = u.reshape(3,)
    v = v.reshape(3,)
    c = np.dot(u,v)/np.linalg.norm(u)/np.linalg.norm(v) # Cos of the angle
    angle = np.arccos(np.clip(c, -1, 1)) # Radian angle
    return angle

In [11]:
def angleIndex(u,v):
    # Based on MvG's script, May 14 2013
    # Return angle between index-mapped vectors on the \Pi-plane
    # (-180,180) degrees, relative to u
    # For left-handed coordinate system (i.e. x right, y down, CW)
    u = u.reshape(2,) # [x,y]
    v = v.reshape(2,) # [x,y]
    dot = u[0]*v[0] + u[1]*v[1]
    det = u[0]*v[1] - u[1]*v[0]
    angle = np.arctan2(det,dot) # Angle in radians
    return angle

In [11]:
def indexRotator(vector, theta, origin = [0,0]): 
    # Rotate x,y CCW around xo,yo by theta (rad)
    # Defaults to origin at (0,0)
    xo = origin[0]; yo = origin[1]
    x = vector[0]; y = vector[1]
    xr=np.cos(theta)*(x-xo)-np.sin(theta)*(y-yo) + xo
    yr=np.sin(theta)*(x-xo)+np.cos(theta)*(y-yo) + yo
    result = np.array([xr,yr]).reshape(2,1)
    return result

In [120]:
def APiToOPi(stress, resolution):
    center = int(np.floor(resolution/2))
    vec = np.zeros((2,1))
    vec[0] = stress[0] - center
    vec[1] = center - stress[1]
    return vec

In [121]:
def OPiToAPi(stress,resolution):
    center = int(np.floor(resolution/2))
    vec = np.zeros((2,1))
    vec[0] = stress[0] + center
    vec[1] = center - stress[1]
    return vec

In [23]:
def stressToIndex(stress, res_x, res_y, R, sliced=True):
    #===============================================================
    # Maps principal normal stress to A\Pi-plane index notation
    # (Array Ready Deviatoric Index Space)
    # [s_1, s_2, s_3].T -> [i,j].T
    # Justin Bonus, Aug 5 2019
    # 
    # Output can be floating point or integer
    # Remember that stresses are scaled down by sqrt(2/3) 
    # when projected onto \pi-plane.
    #===============================================================

    dev_stress = dev(stress) # Projects stress onto \pi-plane (if not already)
    
    # Define index axis
    unit_x = (np.array([[0],[-np.sqrt(1/2)],[np.sqrt(1/2)]])) # Stress-centered X-index axis
    unit_y = (np.array([[-np.sqrt(3/2)+np.sqrt(3/2)/3],[np.sqrt(3/2)/3],[np.sqrt(3/2)/3]])) # Stress-centered Y-index axis
    cen_x = int(res_x/2) # X-center of grid
    cen_y = int(res_y/2) # Y-center of grid
    step_x = 2*R/res_x # Length of a x grid-cell
    step_y = 2*R/res_y # Length of a y grid-cell 
    
    # Project stress onto index axis'
    x_part = -innerProduct(dev_stress,unit_x,1) #range(-R,R)
    y_part = innerProduct(dev_stress,unit_y,1) #range(-R,R)   

    # Floating index space
    i = (x_part/step_x) + cen_x
    j = (y_part/step_y) + cen_y
    
    if sliced == True:
        # Convert to integer index space (Default)
        i = int(np.round(i))
        j = int(np.round(j))
    
    # Bound within array size
    if i > res_x-1:
        i = res_x-1
    if j > res_y-1:
        j = res_y-1
    if i < 0:
        i = 0
    if j < 0:
        j = 0
    
    index = np.array([i,j]).reshape(2,1)
    
    return index

In [5]:
def spaceVis():
    %matplotlib notebook
    %matplotlib notebook
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.widgets import Slider, Button, RadioButtons

    def update(val):
        # Update Resolution from slider
        Res = sRes.val
        # Clear and repopulate axes
        axes.clear()
        draw()	

        # Draw new resolution grid
        for r in range(0,int(Res)+1):
            xline = [-R,R]
            yline = [-R + r*2*R/(Res),-R + r*2*R/(Res)]
            axes.plot(xline, yline, color='black', linestyle='--')
            yline = [-R,R]
            xline = [-R + r*2*R/(Res),-R + r*2*R/(Res)]
            axes.plot(xline, yline, color='black', linestyle='--')

        fig.canvas.draw_idle()

    def draw():
        # Bounding Surface
        axes.plot(xx,yy, color='black',label='Bounding Surface', linewidth=3)

        # Principle stress axis
        axes.arrow(0,0,0,1.2*R,width=.003*R,head_width=.005,head_length=.005, color='black',linestyle=':')
        axes.arrow(0,0,1.4*np.sqrt(2/3)*R,-np.sqrt(1/3)*R,width=.003*R,head_width=.005,head_length=.005, color='black',linestyle='--')
        axes.arrow(0,0,1.4*-np.sqrt(2/3)*R,-np.sqrt(1/3)*R,width=.003*R,head_width=.005,head_length=.005, color='black',linestyle='--')
        axes.text(0, 1.2*R, '$\sigma_1\'$', color = 'black')
        axes.text(1.2*np.sqrt(2/3)*R, -np.sqrt(1/3)*R, '$\sigma_2\'$', color = 'black')
        axes.text(1.2*-np.sqrt(2/3)*R, -np.sqrt(1/3)*R, '$\sigma_3\'$', color = 'black')

        # Orgin \Pi Axis
        axes.text(0,0,'O$\Pi$',fontsize=18)
        axes.arrow(0,0,R/2,0,width=.005*R,head_width=.05*R,head_length=.05*R, color='black')
        axes.arrow(0,0,0,R/2,width=.005*R,head_width=.05*R,head_length=.05*R, color='black')
        axes.text(0, R/2 + 0.05*R, '$x$', color = 'black')
        axes.text(R/2 + 0.05*R, 0, '$y$', color = 'black')

        # Array \Pi Axis
        axes.text(-R,R, 'A$\Pi$', fontsize=18)
        axes.arrow(-R,R,R/2,0,width=.005*R,head_width=.05*R,head_length=.05*R, color='black')
        axes.arrow(-R,R,0,-R/2,width=.005*R,head_width=.05*R,head_length=.05*R, color='black')
        axes.text(-R/2 + 0.05*R, R, '$i$', color = 'black')
        axes.text(-R - 0.05*R, R/2 - 0.1*R, '$j$', color = 'black')

        # Axis settings
        axes.set_xlim(-(5/4)*R,(5/4)*R)
        axes.set_xticks([-R,0,R])
        axes.set_xticklabels(['R','0','R'])
        axes.set_ylim(-(5/4)*R,(5/4)*R)
        axes.set_yticks([-R,0,R])
        axes.set_yticklabels(['R','0','R'])
        axes.grid(False)

        plt.show()


    # ---------------------------------------------------------------------

    R = 0.1 # Initial bounding surface radius
    Res0 = 10 # Initial Resolution

    # Set up figure
    fig, axes = plt.subplots(1,1, figsize=(8,8))

    # Bounding surface coordinates
    rr = np.linspace(0,2*np.pi,360)
    xx = R*np.sin(rr)
    yy = R*np.cos(rr)

    # Set up resolution slider
    axcolor = 'white'
    axRes = plt.axes([0.2, 0.9, 0.65, 0.03], facecolor=axcolor)
    sRes = Slider(axRes, 'Resolution', 1, 100, valinit=Res0, valstep=1)
    sRes.on_changed(update)

    # Draw initial resolution grid
    for r in range(0,Res0+1):
        xline = [-R,R]
        yline = [-R + r*2*R/(Res0),-R + r*2*R/(Res0)]
        axes.plot(xline, yline, color='black', linestyle='--')
        yline = [-R,R]
        xline = [-R + r*2*R/(Res0),-R + r*2*R/(Res0)]
        axes.plot(xline, yline, color='black', linestyle='--')

    # Draw axes
    draw()

    plt.show()

In [29]:
def psiSearch(dStrain, CurStress, res_x, res_y, R):
    # ==================================================================
    # Identify indices of \psi values on the input dStrain line
    # Currently using Bresenham line formulation for efficiency
    # Will never return indexes that exceed the resolution, but 
    # may be 'compressed'/'clipped' to fit
    #
    # INPUT:
    # dStrain: 3x1 strain differential
    # CurStress: 2x1 indexed stress state, UL-centered
    # res_x, res_y: \Pi-plane index resolution
    # R: Radius of Von-Mises cylinder/extent of index space
    # ==================================================================
    #from skimage.draw import line

    # Find index-space center
    cen_x = int(np.floor(res_x/2))
    cen_y = cen_x
    rad = cen_y
    
    # Process stress state
    dev_dStrain = dev(dStrain)
    norm_dev_dStrain = normS(dev_dStrain)
    small=10e-11 # Tolerance to prevent zero vector issues

   # Check if dev_dStrain is zero vector AKA directionless
    if norm_dev_dStrain < small:
        indexCurStress = CurStress - rad # Map to O\Pi index space
        ext_dev_dStrain = np.array([[indexCurStress[0] + rad],[indexCurStress[1] + rad]])
 
    else:
        # Map to center-centered index space
        indexCurStress = CurStress - rad # Map to O\Pi index space
        indexDStrain = stressToIndex(dev_dStrain, res_x, res_y, R, sliced=False) - rad # Map to O\Pi index space

        east = np.array([[1],[0]])
        theta = angleIndex(east, indexDStrain)
        rotIndexCurStress = indexRotator(indexCurStress, -theta)
        ext_dev_dStrain = np.array([[rad*np.sin(np.arccos(rotIndexCurStress[1]/rad))], 
                                    [rotIndexCurStress[1]]])
        ext_dev_dStrain = indexRotator(ext_dev_dStrain.reshape(2,1), theta)        
        if np.isnan(ext_dev_dStrain).any():
            ext_dev_dStrain = indexCurStress
        ext_dev_dStrain = ext_dev_dStrain + rad # Upperleft-recenter

    # Map CurStress to upperleft-centered index space
    indexCurStress = indexCurStress + rad

    # Create Bresenham line coordinates
    rr, cc = line(int(np.round(indexCurStress[1])), int(np.round(indexCurStress[0])), int(np.round(ext_dev_dStrain[1])), int(np.round(ext_dev_dStrain[0])))

    # Constrains values attempting to exit cartesian grid
    #rr = rr.clip(0, res_x-1)
    #rr = cc.clip(0, res_x-1)
    
    return rr, cc

In [16]:
def psiResiduals(CurStress, Stress0, dStrain, kappaMatrix):
    # Author: Justin Bonus, Dec. 18, 2019
    # psiResiduals() plots the associated residuals of each coordinate
    # relative to an input stress state.
    #
    # stress: A\Pi coordinate of active stress
    # Stress0: A\Pi coordinate of unloading stress, already north-bound
    # kappaMatrix: Solution matrice of desired size
    
    # Enter deviatoric plane
    dev_CurStress = dev(CurStress)
    dev_NextStress0 = dev(Stress0)
    dev_dStrain = dev(dStrain)
    
    # Process resolution parameters
    res_x, res_y, cen_x, cen_y = resVals(kappaMatrix.shape)
    
    # Map to float index space
    indexCurStress = stressToIndex(dev_CurStress, res_x, res_y, RR, sliced=False) # Float index
    indexNextStress0 = stressToIndex(dev_NextStress0, res_x, res_y, RR, sliced=False) # Float index
    indexCurStress = APiToOPi(indexCurStress, res_x) # O\Pi
    indexNextStress0 = APiToOPi(indexNextStress0, res_x) # O\Pi

    # Find north angle
    north = np.array([0,1]).reshape(2,1) # O\Pi
    if np.all(indexNextStress0 == 0):
        northAngle = 0. # If zero vector stress, set angles to 0
    else:
        northAngle = angleIndex(north, indexNextStress0) # Angle of unloading stress from north
    
    # Rotated float index stress-state (O\Pi)
    indexNorthCurStress = indexRotator(indexCurStress, -northAngle)
    indexNorthNextStress0 = indexRotator(indexNextStress0, -northAngle)
    
    # Convert to A\Pi
    indexNorthCurStress = OPiToAPi(indexNorthCurStress, res_x)
    indexNorthNextStress0 = OPiToAPi(indexNorthNextStress0, res_x)
    
    # Retrieve appropiate layer from solution matrix
    layerK = (cen_y) - int(np.round(indexNorthNextStress0[1]))
    cartK = np.array(kappaMatrix[:,:,layerK])
    
    # Find current coordinate values
    CurKappa = cartK[int(indexNorthCurStress[1]),int(indexNorthCurStress[0])]
    CurH = (CurKappa*hh)**(mm)
    
    # Calculate residuals at all coordinates
    for j in range(0, res_y):
        for i in range(0, res_x):
            NextKappa = cartK[j,i] # H' value of NextStress cell
            NextH = (NextKappa*hh)**(mm) # Compute hardening moduli
            psi = (2*G) / (1 + 3*G*((1-beta)/NextH + beta/NextH)) # \psi value of specific \psi-H' pair
            
            # Solve base equation, zero res_R means pair matches true function (i.e. no error)
            res_R = np.abs(vectorNorm(dev_CurStress + psi*dev_dStrain + NextKappa*(dev_CurStress + psi*dev_dStrain - dev_NextStress0),1) - R)
            cartK[j,i] = res_R
    
    # Value extent of residuals
    minval = cartK.min()
    maxval = cartK.max()
    
    # Create figure for plotting results
    import matplotlib.cm as cm
    import matplotlib.colors as mcolors
    fig, ax = plt.subplots(figsize=(10,10))
    
    def colorbar_index(ncolors, cmap, minval, maxval):
        cmap = cmap_discretize(cmap, ncolors)
        mappable = cm.ScalarMappable(cmap=cmap)
        mappable.set_array([])
        mappable.set_clim(-0.5, ncolors+0.5)
        colorbar = plt.colorbar(mappable)
        colorbar.set_ticks(np.linspace(0, ncolors, ncolors))
        colorbar.set_ticklabels(np.trunc(np.linspace(0, maxval, 11)))

    def cmap_discretize(cmap, N):
        # Return a discrete colormap from the continuous colormap cmap.
        #    cmap: colormap instance, eg. cm.jet. 
        #    N: number of colors.

        if type(cmap) == str:
            cmap = plt.get_cmap(cmap)
        colors_i = np.concatenate((np.linspace(0, 1., N), (0.,0.,0.,0.)))
        colors_rgba = cmap(colors_i)
        indices = np.linspace(0, 1., N+1)
        cdict = {}
        for ki,key in enumerate(('red','green','blue')):
            cdict[key] = [ (indices[i], colors_rgba[i-1,ki], colors_rgba[i,ki]) for i in range(N+1) ]
        # Return colormap object.
        return mcolors.LinearSegmentedColormap(cmap.name + "_%d"%N, cdict, 1024)
    
    # Create search-line coordinates
    rotDStrain = quatHydroRotator(dev_dStrain, northAngle) # Rotate strain appropiately
    rr, cc = psiSearch(rotDStrain, indexNorthCurStress, res_x, res_y, R) # Retrieve coordinate lists
    cartK[rr, cc] = maxval # Set coordinate values so they are visually apparent
    
    # Plot
    cmap = plt.get_cmap('RdYlGn_r') # Set color-scheme
    ax.imshow(cartK, cmap=cmap, norm=matplotlib.colors.LogNorm()) # Plot use log scale for values
    colorbar_index(ncolors=11, cmap=cmap, minval=minval, maxval=maxval) # Add legend
    ax.set_title('Search Residuals')
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    
    plt.show()

In [275]:
def boundingCoordinate(dStrain, CurStress, indexR):
    # Find the coordinate of a strain search line intersection on a 
    # bounding surface relative to current stress. Keep lightweight
    #
    # dStrain: Strain Increment, 2x1 O\Pi
    # CurStress: Current Stress, 2x1 O\Pi
    # indexR: Bounding Surface Radius, index integer
    #
    # bC: Coordinate of BS intersection, 2x1 O\Pi float
    
    # Variables
    a = float(dStrain[0])
    b = float(dStrain[1])
    c = float(CurStress[0])
    d = float(CurStress[1])
    R = float(indexR)
    
    # Check if there are zero components for dStrain
    small = 1e-8 # Placeholder for '0', keeps equations usable
    if np.all(dStrain==0):
        bC = np.array([c,d]).reshape(2,1)
        return bC
    if np.sign(a) == 0:
        if np.sign(b) == 1:
            a = -small
        elif np.sign(b) == -1:
            a = small
        else:
            a = small
    

    # Strain Slope
    m = b/a
    
    # Two intersection points on the Boundin Surface are found for a 
    # line following dStrain. Choose appropiate one via slope's direction.
    if np.sign(a) == 1:
        x = (np.sqrt(-c**2 * m**2 + 2*c*d*m - d**2 + (m**2 + 1)*R**2) + c*m**2 - d*m) / (m**2 + 1)
        y = (m*np.sqrt(-c**2 * m**2 + 2*c*d*m - d**2 + (m**2 +1)*R**2) - c*m**2 + d) / (m**2 + 1)
    elif np.sign(a) == -1:
        x = -1*(np.sqrt(-c**2 * m**2 + 2*c*d*m - d**2 + (m**2 + 1)*R**2) - c*m**2 + d*m) / (m**2 + 1)
        y = (d - m*(np.sqrt(-c**2 * m**2 + 2*c*d*m - d**2 + (m**2 +1)*R**2) + c)) / (m**2 + 1)
    
    # Bounding Surface coordinate (2x1, O\Pi float)
    bC = np.array([x,y]).reshape(2,1)
    
    return bC

In [260]:
def psiSearch_lite(dStrain, CurStress, res):
    # ==================================================================
    # Identify indices of \psi values on the input dStrain line
    # Currently using Bresenham line formulation for efficiency
    # Will never return indexes that exceed the resolution, but 
    # may be 'compressed'/'clipped' to fit
    #
    # INPUT:
    # dStrain: 2x1 strain differential, A\Pi
    # CurStress: 2x1 indexed stress state, A\Pi
    # res: \Pi-plane index resolution
    #
    # OUTPUT:
    # rr: y-coords for Bresenham line
    # cc: x-coords for Bresenham line
    # ==================================================================
    #from skimage.draw import line

    # Find index-space center
    cen = int(np.floor(res/2))

    # Map to O\Pi
    CurStressO = APiToOPi(CurStress, res)
    
    # Calculate coordinate of BS intersection for O\Pi space
    bCoord = boundingCoordinate(dStrain, CurStressO, cen)
    
    # Map to A\Pi
    bCoord = OPiToAPi(bCoord, res)

    # Variable
    cx = int(np.round(CurStress[0]))
    cy = int(np.round(CurStress[1]))
    bx = int(np.round(bCoord[0]))
    by = int(np.round(bCoord[1]))
    
    # Create Bresenham line coordinates in A\Pi space
    rr, cc = line(cy, cx, by, bx)

    # Constrains values attempting to exit cartesian grid
    #rr = rr.clip(0, res_x-1)
    #cc = cc.clip(0, res_x-1)
    
    return rr, cc